In [1]:
import numpy as np
import os
import pandas as pd
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
# import seaborn as sns
import pickle
import time
import gc
from tqdm import tqdm, tqdm_notebook

%matplotlib inline

#每次可以输出多个变量
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from pylab import rcParams
rcParams['figure.figsize'] = 14, 6

import warnings
warnings.filterwarnings("ignore")

#中文字体
import matplotlib
matplotlib.use('qt4agg')
#指定默认字体
matplotlib.rcParams['font.sans-serif'] = ['SimHei']
matplotlib.rcParams['font.family'] = 'sans-serif'
#解决负号'-'显示为方块的问题
matplotlib.rcParams['axes.unicode_minus'] = False

In [2]:
df = pd.read_csv('./data/df_preprocess.csv')

df.shape
df.head()

(1041674, 72)

,instance_id,time,city,province,carrier,devtype,make,model,nnt,os,osv,os_name,adid,advert_id,orderid,campaign_id,creative_id,creative_tp_dnf,app_cate_id,app_id,inner_slot_id,creative_type,creative_width,creative_height,creative_is_jump,creative_is_download,creative_is_js,creative_is_voicead,creative_has_deeplink,app_paid,advert_name,click,advert_industry_inner_split_0,advert_industry_inner_split_1,user_tags_0,user_tags_1,user_tags_2,user_tags_3,user_tags_4,user_tags_5,user_tags_6,user_tags_7,user_tags_8,user_tags_9,user_tags_10,user_tags_11,user_tags_12,user_tags_13,user_tags_14,user_tags_15,user_tags_16,user_tags_17,user_tags_18,user_tags_19,user_tags_20,user_tags_21,user_tags_22,user_tags_23,user_tags_24,user_tags_25,user_tags_26,user_tags_27,user_tags_28,user_tags_29,user_tags_30,user_tags_31,user_tags_32,user_tags_33,user_tags_34,user_tags_35,user_tags_36,user_tags_37
0,86294719979897807,2190219034,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1280,720,1,0,0,0,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2699289844928136052,2190221070,2,2,2,1,2,2,1,1,2,1,2,1,2,1,2,2,2,2,2,1,960,640,1,0,0,0,0,0,1,0,1,1,1,1,1,0,1,0,1,0,0,0,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
2,3117527168445845752,2190219793,3,3,2,1,3,3,1,1,3,1,3,2,3,2,3,3,3,3,3,1,960,640,1,0,0,0,0,0,2,0,2,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,3398484891050993371,2190221704,4,1,3,1,4,4,1,1,4,1,4,3,4,3,4,4,3,4,4,2,1280,720,1,0,0,0,0,0,3,0,3,3,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
4,2035477570591176488,2190220024,5,1,1,1,5,5,2,2,5,2,5,1,5,1,5,2,4,5,5,1,960,640,1,0,0,0,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [3]:
df = df.sort_values(by=['time'], ascending=True)

In [4]:
def timestamp_datetime(value):
    format = '%Y-%m-%d %H:%M:%S'
    value = time.localtime(value)
    dt = time.strftime(format, value)
    return dt


def convert_data(data):
    data['time'] = data.time.apply(timestamp_datetime)
    data['day'] = data.time.apply(lambda x: int(x[8:10]))
    data['hour'] = data.time.apply(lambda x: int(x[11:13]))
    data['minute'] = data.time.apply(lambda x: int(x[14:15])) * 10
    return data


# train 6.1 6.2 test 6.3
df = convert_data(df)

In [5]:
df_user = pd.read_csv('./feature/user_id.csv')

In [6]:
df = pd.concat([df, df_user], axis=1)
df.shape

(1041674, 78)

## 预处理

In [7]:
from collections import Counter

In [8]:
# 去除出现次数少的
make_most = [
    i[0]
    for i in sorted(
        Counter(df.make).items(), key=lambda x: x[1], reverse=True)[:210]
]
model_most = [
    i[0]
    for i in sorted(
        Counter(df.model).items(), key=lambda x: x[1], reverse=True)[:1150]
]
adid_most = [
    i[0]
    for i in sorted(
        Counter(df.adid).items(), key=lambda x: x[1], reverse=True)[:650]
]
orderid_most = [
    i[0]
    for i in sorted(
        Counter(df.orderid).items(), key=lambda x: x[1], reverse=True)[:530]
]
creative_id_most = [
    i[0]
    for i in sorted(
        Counter(df.creative_id).items(), key=lambda x: x[1], reverse=True)[:
                                                                           350]
]
app_id_most = [
    i[0]
    for i in sorted(
        Counter(df.app_id).items(), key=lambda x: x[1], reverse=True)[:150]
]
inner_slot_id_most = [
    i[0]
    for i in sorted(
        Counter(df.inner_slot_id).items(), key=lambda x: x[1], reverse=True)
    [:260]
]

In [9]:
df.make = df.make.apply(lambda x: x if x in make_most else 9999)
df.model = df.model.apply(lambda x: x if x in model_most else 9999)
df.adid = df.adid.apply(lambda x: x if x in adid_most else 9999)
df.orderid = df.orderid.apply(lambda x: x if x in orderid_most else 9999)
df.creative_id = df.creative_id.apply(
    lambda x: x if x in creative_id_most else 9999)
df.app_id = df.app_id.apply(lambda x: x if x in app_id_most else 9999)
df.inner_slot_id = df.inner_slot_id.apply(
    lambda x: x if x in inner_slot_id_most else 9999)

In [10]:
# his = df[df.time < '2039-06-01 00:00:00']
# df = df[df.time >= '2039-06-01 00:00:00']

# his.shape
# df.shape

# his['noclick'] = 1 - his.click

In [11]:
ad_col = [
    'adid', 'advert_id', 'orderid', 'campaign_id', 'creative_id',
    'creative_tp_dnf', 'creative_type', 'advert_name',
    'advert_industry_inner_split_0', 'advert_industry_inner_split_1'
]
ad_cate_col = [
    'creative_is_jump', 'creative_is_download', 'creative_is_js',
    'creative_is_voicead', 'creative_has_deeplink'
]
medim_col = ['app_cate_id', 'app_id', 'inner_slot_id', 'app_paid']
user_col = [
    'user_tags_0', 'user_tags_1', 'user_tags_2', 'user_tags_3', 'user_tags_4',
    'user_tags_5', 'user_tags_6', 'user_tags_7', 'user_tags_8', 'user_tags_9',
    'user_tags_10', 'user_tags_11', 'user_tags_12', 'user_tags_13',
    'user_tags_14', 'user_tags_15', 'user_tags_16', 'user_tags_17',
    'user_tags_18', 'user_tags_19', 'user_tags_20', 'user_tags_21',
    'user_tags_22', 'user_tags_23', 'user_tags_24', 'user_tags_25',
    'user_tags_26', 'user_tags_27', 'user_tags_28', 'user_tags_29',
    'user_tags_30', 'user_tags_31', 'user_tags_32', 'user_tags_33',
    'user_tags_34', 'user_tags_35', 'user_tags_36', 'user_tags_37'
]
context_col = [
    'city', 'province', 'carrier', 'devtype', 'make', 'model', 'nnt', 'os',
    'osv', 'os_name'
]

cont_col = ['creative_width', 'creative_height']
by_col = ['instance_id', 'time']
target = ['click']

cross_col = ad_col + ad_cate_col + medim_col + user_col + context_col
full_col = ad_col + ad_cate_col + medim_col + user_col + context_col + cont_col + by_col + target
len(full_col)

72

## user特征

In [18]:
df = df.drop(['user_id'], axis=1)
df = df.rename(index=str, columns={'user_id2':'user_id'})

In [22]:
#第几次点击
from collections import defaultdict, Counter
import ast

for i in tqdm_notebook(ad_col + medim_col + context_col):
    temp = defaultdict(lambda: defaultdict(lambda: -1))
    
    def user_count(x, i):
        temp[x.user_id][x[i]] += 1
        return temp[x.user_id][x[i]]

    df[i + '_count'] = df.apply(user_count, args=(i, ), axis=1)

A Jupyter Widget

In [23]:
#上下次浏览时间差
from datetime import datetime, timedelta

for i in tqdm_notebook(ad_col + medim_col + context_col):
    df[i + '_next_count'] = df[i + '_count'] + 1
    df[i + '_last_count'] = df[i + '_count'] - 1
    temp_time = df[[i, i + '_count', 'time']].copy()
    
    temp_time.columns = [i, i + '_next_count', i + '_next_time']
    df = df.merge(temp_time, on=[i, i + '_next_count'], how='left')
    temp_time.columns = [i, i + '_last_count', i + '_last_time']
    df = df.merge(temp_time, on=[i, i + '_last_count'], how='left')


    def next_time_sub(x):
        if type(x[i + '_next_time']) == float:
            return 99999
        else:
            return (datetime.strptime(x[i + '_next_time'], '%Y-%m-%d  %H:%M:%S') -
                    datetime.strptime(x.time, '%Y-%m-%d  %H:%M:%S')).seconds
    
    
    def last_time_sub(x):
        if type(x[i + '_last_time']) == float:
            return 99999
        else:
            return (
                datetime.strptime(x.time, '%Y-%m-%d  %H:%M:%S') -
                datetime.strptime(x[i + '_last_time'], '%Y-%m-%d  %H:%M:%S')).seconds
    
    
    df[i + '_next_time_sub'] = df.apply(next_time_sub, axis=1)
    df[i + '_last_time_sub'] = df.apply(last_time_sub, axis=1)
    
    del temp_time
    _ = gc.collect()

A Jupyter Widget

MemoryError: 

In [ ]:
# single第几次点击
for i in tqdm_notebook(ad_col + medim_col + context_col):
    temp = defaultdict(lambda: -1)
    
    def user_count(x):
        temp[x] += 1
        return temp[x]

    df[i + '_single_count'] = df[i].apply(user_count)

In [ ]:
#上下次浏览时间差
from datetime import datetime, timedelta

for i in tqdm_notebook(ad_col + medim_col + context_col):
    df[i + '_single_next_count'] = df[i + '_single_count'] + 1
    df[i + '_single_last_count'] = df[i + '_single_count'] - 1
    temp_time = df[[i, i + '_single_count', 'time']].copy()
    
    temp_time.columns = [i, i + '_single_next_count', i + '_next_time']
    df = df.merge(temp_time, on=[i, i + '_single_next_count'], how='left')
    temp_time.columns = [i, i + '_single_last_count', i + '_last_time']
    df = df.merge(temp_time, on=[i, i + '_single_last_count'], how='left')


    def next_time_sub(x):
        if type(x[i + '_single_next_time']) == float:
            return 99999
        else:
            return (datetime.strptime(x[i + '_single_next_time'], '%Y-%m-%d  %H:%M:%S') -
                    datetime.strptime(x.time, '%Y-%m-%d  %H:%M:%S')).seconds
    
    
    def last_time_sub(x):
        if type(x[i + '_single_last_time']) == float:
            return 99999
        else:
            return (
                datetime.strptime(x.time, '%Y-%m-%d  %H:%M:%S') -
                datetime.strptime(x[i + '_single_last_time'], '%Y-%m-%d  %H:%M:%S')).seconds
    
    
    df[i + '_single_next_time_sub'] = df.apply(next_time_sub, axis=1)
    df[i + '_single_last_time_sub'] = df.apply(last_time_sub, axis=1)
    
    del temp_time
    _ = gc.collect()

In [30]:
df.shape
df.head()

(348536, 219)

,instance_id,time,city,province,carrier,devtype,make,model,nnt,os,osv,os_name,adid,advert_id,orderid,campaign_id,creative_id,creative_tp_dnf,app_cate_id,app_id,inner_slot_id,creative_type,creative_width,creative_height,creative_is_jump,creative_is_download,creative_is_js,creative_is_voicead,creative_has_deeplink,app_paid,advert_name,click,advert_industry_inner_split_0,advert_industry_inner_split_1,user_tags_0,user_tags_1,user_tags_2,user_tags_3,user_tags_4,user_tags_5,user_tags_6,user_tags_7,user_tags_8,user_tags_9,user_tags_10,user_tags_11,user_tags_12,user_tags_13,user_tags_14,user_tags_15,user_tags_16,user_tags_17,user_tags_18,user_tags_19,user_tags_20,user_tags_21,user_tags_22,user_tags_23,user_tags_24,user_tags_25,user_tags_26,user_tags_27,user_tags_28,user_tags_29,user_tags_30,user_tags_31,user_tags_32,user_tags_33,user_tags_34,user_tags_35,user_tags_36,user_tags_37,day,hour,minute,adid_day_query,adid_day_posrate,advert_id_day_query,advert_id_day_posrate,orderid_day_query,orderid_day_posrate,campaign_id_day_query,campaign_id_day_posrate,creative_id_day_query,creative_id_day_posrate,creative_tp_dnf_day_query,creative_tp_dnf_day_posrate,creative_type_day_query,creative_type_day_posrate,advert_name_day_query,advert_name_day_posrate,advert_industry_inner_split_0_day_query,advert_industry_inner_split_0_day_posrate,advert_industry_inner_split_1_day_query,advert_industry_inner_split_1_day_posrate,app_cate_id_day_query,app_cate_id_day_posrate,app_id_day_query,app_id_day_posrate,inner_slot_id_day_query,inner_slot_id_day_posrate,app_paid_day_query,app_paid_day_posrate,city_day_query,city_day_posrate,province_day_query,province_day_posrate,carrier_day_query,carrier_day_posrate,devtype_day_query,devtype_day_posrate,make_day_query,make_day_posrate,model_day_query,model_day_posrate,nnt_day_query,nnt_day_posrate,os_day_query,os_day_posrate,osv_day_query,osv_day_posrate,os_name_day_query,os_name_day_posrate,adid_day_hour_query,adid_day_hour_posrate,advert_id_day_hour_query,advert_id_day_hour_posrate,orderid_day_hour_query,orderid_day_hour_posrate,campaign_id_day_hour_query,campaign_id_day_hour_posrate,creative_id_day_hour_query,creative_id_day_hour_posrate,creative_tp_dnf_day_hour_query,creative_tp_dnf_day_hour_posrate,creative_type_day_hour_query,creative_type_day_hour_posrate,advert_name_day_hour_query,advert_name_day_hour_posrate,advert_industry_inner_split_0_day_hour_query,advert_industry_inner_split_0_day_hour_posrate,advert_industry_inner_split_1_day_hour_query,advert_industry_inner_split_1_day_hour_posrate,app_cate_id_day_hour_query,app_cate_id_day_hour_posrate,app_id_day_hour_query,app_id_day_hour_posrate,inner_slot_id_day_hour_query,inner_slot_id_day_hour_posrate,app_paid_day_hour_query,app_paid_day_hour_posrate,city_day_hour_query,city_day_hour_posrate,province_day_hour_query,province_day_hour_posrate,carrier_day_hour_query,carrier_day_hour_posrate,devtype_day_hour_query,devtype_day_hour_posrate,make_day_hour_query,make_day_hour_posrate,model_day_hour_query,model_day_hour_posrate,nnt_day_hour_query,nnt_day_hour_posrate,os_day_hour_query,os_day_hour_posrate,osv_day_hour_query,osv_day_hour_posrate,os_name_day_hour_query,os_name_day_hour_posrate,adid_day_hour_minute_query,adid_day_hour_minute_posrate,advert_id_day_hour_minute_query,advert_id_day_hour_minute_posrate,orderid_day_hour_minute_query,orderid_day_hour_minute_posrate,campaign_id_day_hour_minute_query,campaign_id_day_hour_minute_posrate,creative_id_day_hour_minute_query,creative_id_day_hour_minute_posrate,creative_tp_dnf_day_hour_minute_query,creative_tp_dnf_day_hour_minute_posrate,creative_type_day_hour_minute_query,creative_type_day_hour_minute_posrate,advert_name_day_hour_minute_query,advert_name_day_hour_minute_posrate,advert_industry_inner_split_0_day_hour_minute_query,advert_industry_inner_split_0_day_hour_minute_posrate,advert_industry_inner_split_1_day_hour_minute_query,advert_industry_inner_split_1_day_hour_minute_posrate,app_cate_id_day_hour_minute_query,app_cat

In [33]:
df.to_csv('./feature/feature_nonoise/time_feature.csv',index=None)